In [1]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import requests
from datetime import datetime
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
retry_session = retry(requests.Session(), retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
today = datetime.now().strftime("%Y-%m-%d")
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [38.8951, 51.5085, 35.6895, 21.0245],
	"longitude": [-77.0364, -0.1257, 139.6917, 105.8412],
	"start_date": "2020-01-01",
	"end_date": today,
	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
	"hourly": ["temperature_2m", "precipitation", "weather_code", "relative_humidity_2m", "wind_speed_10m"]
}
responses = openmeteo.weather_api(url, params=params)

In [4]:
city = ["Washington", "London", "Tokyo", "HaNoi"]
all_data = []
for idx, response in enumerate(responses):
    Latitude = response.Latitude()
    Longitude = response.Longitude()
    Elevation = response.Elevation()
    data = {
        "City": city[idx],
        "Latitude": Latitude,
        "Longitude": Longitude,
        "Elevation": Elevation
    }    
    all_data.append(data)

df = pd.DataFrame(all_data)
df

,City,Latitude,Longitude,Elevation
0,Washington,38.910366,-77.072510,12.0
1,London,51.493847,-0.163025,23.0
2,Tokyo,35.676624,139.691116,40.0
3,HaNoi,21.054480,105.807106,10.0


In [5]:
all_hourly_data = []
for idx, response in enumerate(responses):
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
	hourly_weather_code = hourly.Variables(2).ValuesAsNumpy()
	hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}

	hourly_data["city"] = city[idx]
	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["precipitation"] = hourly_precipitation
	hourly_data["weather_code"] = hourly_weather_code
	hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
	hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

	df = pd.DataFrame(hourly_data)
	all_hourly_data.append(df)

hourly_df = pd.concat(all_hourly_data)
hourly_df.head()

,date,city,temperature_2m,precipitation,weather_code,relative_humidity_2m,wind_speed_10m
0,2020-01-01 00:00:00+00:00,Washington,7.45,0.0,3.0,55.246738,9.726665
1,2020-01-01 01:00:00+00:00,Washington,7.25,0.0,3.0,53.597992,8.699793
2,2020-01-01 02:00:00+00:00,Washington,6.75,0.0,3.0,49.277168,12.287555
3,2020-01-01 03:00:00+00:00,Washington,6.45,0.0,3.0,49.375435,18.864824
4,2020-01-01 04:00:00+00:00,Washington,6.15,0.0,3.0,54.093021,17.826363


In [6]:
hourly_df.describe()

,temperature_2m,precipitation,weather_code,relative_humidity_2m,wind_speed_10m
count,210240.000000,210240.000000,210240.000000,210240.000000,210240.000000
mean,16.555178,0.170149,11.793341,74.781738,12.000031
std,9.118156,0.804178,20.742167,17.197201,7.266247
min,-14.500000,0.000000,0.000000,11.150274,0.000000
25%,9.500000,0.000000,0.000000,62.927370,6.927077
50%,16.900000,0.000000,3.000000,78.235580,10.440000
75%,24.049999,0.000000,3.000000,89.129372,15.307410
max,41.150002,29.000000,75.000000,100.000000,76.049515


In [7]:
hourly_df.isnull().sum()

date                    0
city                    0
temperature_2m          0
precipitation           0
weather_code            0
relative_humidity_2m    0
wind_speed_10m          0
dtype: int64

In [8]:
all_daily_data = []
for idx, response in enumerate(responses):
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}

	daily_data['city'] = city[idx]
	daily_data["temperature_2m_max"] = daily_temperature_2m_max
	daily_data["temperature_2m_min"] = daily_temperature_2m_min
	daily_data["precipitation_sum"] = daily_precipitation_sum

	df = pd.DataFrame(daily_data)
	all_daily_data.append(df)

daily_df = pd.concat(all_daily_data)
daily_df.head()

,date,city,temperature_2m_max,temperature_2m_min,precipitation_sum
0,2020-01-01 00:00:00+00:00,Washington,7.70,2.80,0.0
1,2020-01-02 00:00:00+00:00,Washington,9.20,-1.85,0.0
2,2020-01-03 00:00:00+00:00,Washington,13.30,7.30,6.6
3,2020-01-04 00:00:00+00:00,Washington,14.35,9.10,10.9
4,2020-01-05 00:00:00+00:00,Washington,9.35,0.80,2.1


In [9]:
daily_df.describe()

,temperature_2m_max,temperature_2m_min,precipitation_sum
count,8760.000000,8760.000000,8760.000000
mean,20.904144,12.783373,4.083585
std,8.805382,8.890944,9.905578
min,-8.000000,-14.500000,0.000000
25%,13.750000,5.750000,0.000000
50%,21.350000,12.900000,0.300000
75%,28.250000,20.299999,3.500000
max,41.150002,29.600000,176.799988


In [10]:
daily_df.isnull().sum()

date                  0
city                  0
temperature_2m_max    0
temperature_2m_min    0
precipitation_sum     0
dtype: int64

In [11]:
load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

hourly_df.to_sql(
    name = "weather_hourly",
    con = engine,
    if_exists = "replace",
    index = False
)

daily_df.to_sql(
    name = "weather_daily",
    con = engine,
    if_exists = "replace",
    index = False
)

print("Load completed")

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)